## Hackathon Copenhagen 2025

This notebook is an introduction to the pydna package, meant to be used as a starting point for the hackathon.

Click on the link below to open the notebook in Google Colab, so that you can   easily run it locally.

<a target="_blank" href="https://colab.research.google.com/github/pydna-group/pydna/blob/master/docs/notebooks/hackathon_copenhagen_2025.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>





The below cell simply installs the necessary dependencies to run pydna and downloads the files to run the code if you are running in google collab. If you are running this in your computer, you will have to install `pydna` from pip or your favourite package manager, and download the files yourself from [here](https://github.com/pydna-group/pydna/tree/master/docs/notebooks).


In [ ]:
%%capture
# Install pydna (only when running on Colab)
import sys
if 'google.colab' in sys.modules:
    %pip install pydna[clipboard,download,express,gel]
    !curl -LO "https://github.com/pydna-group/pydna/raw/master/docs/notebooks/U49845.fasta"
    !curl -LO "https://github.com/pydna-group/pydna/raw/master/docs/notebooks/CU329670.gb"
    !mkdir images/
    !curl -LO "https://github.com/pydna-group/pydna/raw/master/docs/notebooks/images/snapgene_screenshot.png"
    !mv snapgene_screenshot.png images/
    !curl -LO "https://github.com/pydna-group/pydna/raw/master/docs/notebooks/images/snapgene_screenshot2.png"
    !mv snapgene_screenshot2.png images/


## Dseq and Dseqrecord

Pydna contains classes to represent double stranded DNA sequences that can:

* Be linear
* Be circular
* Contain overhangs (sticky ends).

These sequences can be used to simulate molecular biology methods such as cloning and PCR. The main classes used to represent sequences are `Dseq` and `Dseqrecord`.
* `Dseq` represents the sequence only. Think of it as a FASTA file.
* `Dseqrecord` can contain sequence features and other info such as publication, authors, etc. Think of it as a Genbank file.

The most important aspects are shown below:


In [ ]:
from pydna.dseq import Dseq

# We initialize a Dseq object with a string of DNA sequence, by default it is linear
my_dseq = Dseq("ATCG")

# We can specify the circularity of the sequence
my_circ_dseq = Dseq("ATCG", circular=True)

print("Dseq is printed as a string sequence:")
print(my_dseq, "", my_circ_dseq, sep="\n")
print()
print("The representation also prints the circularity and both strands:")
print(repr(my_dseq), "", repr(my_circ_dseq), sep="\n")

print()
print("Circular sequences support circular indexing:", my_circ_dseq[2:1])
print("Linear sequences return an empty sequence object:", my_dseq[2:1])




Dseq is printed as a string sequence:
ATCG

ATCG

The representation also prints the circularity and both strands:
Dseq(-4)
ATCG
TAGC

Dseq(o4)
ATCG
TAGC

Circular sequences support circular indexing: CGA
Linear sequences return an empty sequence object: 


A `Dseqrecord` object contains a `Dseq` object and other information such as the sequence name, description, and features.

In [ ]:
from pydna.dseqrecord import Dseqrecord

# One way to initialize a Dseqrecord object is to pass a Dseq object
my_dseqrecord = Dseqrecord(Dseq("TGCAAACAGTAATGATGGATGAC", circular=False))

# Another way is to pass a string of DNA sequence
my_dseqrecord = Dseqrecord("TGCAAACAGTAATGATGGATGAC", circular=False)

# We can print the Dseqrecord object to see the sequence and other information
print(my_dseqrecord)

# We can also print the sequence only (it is a Dseq object)
print()
print(type(my_dseqrecord.seq), my_dseqrecord.seq)

# We can add features, name it, etc. more on this later.


Dseqrecord
circular: False
size: 23
ID: id
Name: name
Description: description
Number of features: 0
/molecule_type=DNA
Dseq(-23)
TGCAAACAGTAATGATGGATGAC
ACGTTTGTCATTACTACCTACTG

<class 'pydna.dseq.Dseq'> TGCAAACAGTAATGATGGATGAC




> NOTE: The `Dseq` class is a subclass of biopython's `Seq`, whose documentation can be found [here](https://biopython.org/wiki/Seq). `Dseqrecord` is a subclass of biopython's `SeqRecord`, whose documentation can be found [here](https://biopython.org/wiki/SeqRecord).

## Importing Sequence Files

To import files into pydna is simple. pydna provides the `parse` method to read all DNA sequences in a file into a list. As an input, `parse` can take:

* The path to a file from your computer
* A python string with the file content.

The following code shows an example of how to use the `parse` function to import a FASTA file.

In [ ]:
from pydna.parsers import parse

#Import your file into python using its path
file_path = "./U49845.fasta"
files = parse(file_path)

#Show your FASTA file in python
print(files[0].format("fasta"))

>lcl|U49845.1_cds_AAA98665.1_1 [protein=TCP1-beta] [frame=3] [protein_id=AAA98665.1] [location=<1..206] [gbkey=CDS]
TCCTCCATATACAACGGTATCTCCACCTCAGGTTTAGATCTCAACAACGGAACCATTGCC
GACATGAGACAGTTAGGTATCGTCGAGAGTTACAAGCTAAAACGAGCAGTAGTCAGCTCT
GCATCTGAAGCCGCTGAAGTTCTACTAAGGGTGGATAACATCATCCGTGCAAGACCAAGA
ACCGCCAATAGACAACATATGTAA


Note that `parse` returns a `list` object, hence requiring `[0]` to take the first element of the list. When you have a FASTA file that contains multiple sequences, you can index the list accordingly (e.g  `[0]`, `[1]`, ...)

The last line of code uses the `format` method to generate a string representation of the sequence as a FASTA file.

Another example, using a GenBank file ([U49845](https://www.ncbi.nlm.nih.gov/nucleotide/U49845)), is shown below.

In [ ]:
from pydna.parsers import parse

file_path = "./CU329670.gb"
files = parse(file_path)

# Convert the Dseqrecord object into a formatted string in GenBank format
ase1_locus = files[0]
print(ase1_locus)

Dseqrecord
circular: False
size: 4538
ID: CU329670.1
Name: CU329670
Description: Schizosaccharomyces pombe strain 972h- genome assembly, chromosome: I
Database cross-references: BioProject:PRJNA13836, BioSample:SAMEA3138176
Number of features: 15
/molecule_type=DNA
/topology=linear
/data_file_division=PLN
/date=26-APR-2024
/accessions=['CU329670']
/sequence_version=1
/keywords=['']
/source=Schizosaccharomyces pombe (fission yeast)
/organism=Schizosaccharomyces pombe
/taxonomy=['Eukaryota', 'Fungi', 'Dikarya', 'Ascomycota', 'Taphrinomycotina', 'Schizosaccharomycetes', 'Schizosaccharomycetales', 'Schizosaccharomycetaceae', 'Schizosaccharomyces']
/references=[Reference(title='The mitochondrial genome of the fission yeast Schizosaccharomyces pombe: highly homologous introns are inserted at the same position of the otherwise less conserved cox1 genes in Schizosaccharomyces pombe and Aspergillus nidulans', ...), Reference(title='The mitochondrial genome of the fission yeast Schizosaccharomyc

## Importing external sequences

You can use genbank to get sequences from GenBank files. If you need to get sequences from other sources (AddGene, for instance). Let us know and we can help with that.


In [ ]:
from pydna.genbank import Genbank

# Reading the R. cellulolyticum genome from GenBank
gb = Genbank("example@example.com")
genome = gb.nucleotide("CP001348.1")

print(genome)

Dseqrecord
circular: True
size: 4068724
ID: CP001348.1
Name: CP001348
Description: Ruminiclostridium cellulolyticum H10 chromosome, complete genome
Database cross-references: BioProject:PRJNA17419, BioSample:SAMN00623037
Number of features: 7582
/molecule_type=DNA
/topology=circular
/data_file_division=BCT
/date=25-AUG-2017
/accessions=['CP001348', 'AAVC01000000', 'AAVC01000001-AAVC01000121']
/sequence_version=1
/keywords=['']
/source=Ruminiclostridium cellulolyticum H10
/organism=Ruminiclostridium cellulolyticum H10
/taxonomy=['Bacteria', 'Bacillati', 'Bacillota', 'Clostridia', 'Eubacteriales', 'Oscillospiraceae', 'Ruminiclostridium']
/references=[Reference(title='Complete sequence of Clostridium cellulolyticum H10', ...), Reference(title='Direct Submission', ...)]
/comment=URL -- http://www.jgi.doe.gov
JGI Project ID: 4002584
Source DNA and bacteria available from Jizhong Zhou
(jzhou@rccc.ou.edu)
Contacts: Jizhong Zhou (jzhou@rccc.ou.edu)
          David Bruce (microbe@cuba.jgi-psf.o

## Sequence features



Some sequence file formats (like Genbank) include features, describing key biological properties of sequence regions. Let's look at our previous example in SnapGene:

<img src="./images/snapgene_screenshot.png" alt="SnapGene screenshot of the U49845 sequence" style="max-width: 600px;">

<br>

<img src="./images/snapgene_screenshot2.png" alt="SnapGene screenshot of the U49845 sequence" style="max-width: 600px;">

These features can also be seen in the Genbank file. For example, the below text in the Genbank file represents the coding sequence (`CDS`) of the `ase1` gene.

Each feature has:

* A type (e.g. `CDS`)
* A location (e.g. `join(1174..1597,1645..3416)`)
* Optionally, some qualifiers ( key-value pairs, e.g. `gene="ase1"`). Features can have multiple qualifiers of the same type.

```
     CDS             join(1174..1597,1645..3416)
                     /gene="ase1"
                     /locus_tag="SPOM_SPAPB1A10.09"
                     /codon_start=1
                     /product="antiparallel microtubule cross-linking factor
                     Ase1"
                     /protein_id="CAC21482.1"
```

How do these features look like in the `Dseqrecord` object, and how do we access them? Let's go back to our previous example:


In [ ]:
for feature in ase1_locus.features[5:9]:
    print(feature)


type: 3'UTR
location: [393:676](+)
qualifiers:
    Key: locus_tag, Value: ['SPOM_SPAPB1A10.08']

type: gene
location: [1000:3538](+)
qualifiers:
    Key: gene, Value: ['ase1']
    Key: locus_tag, Value: ['SPOM_SPAPB1A10.09']

type: 5'UTR
location: [1000:1173](+)
qualifiers:
    Key: gene, Value: ['ase1']
    Key: locus_tag, Value: ['SPOM_SPAPB1A10.09']

type: CDS
location: join{[1173:1597](+), [1644:3416](+)}
qualifiers:
    Key: codon_start, Value: ['1']
    Key: gene, Value: ['ase1']
    Key: locus_tag, Value: ['SPOM_SPAPB1A10.09']
    Key: product, Value: ['antiparallel microtubule cross-linking factor Ase1']
    Key: protein_id, Value: ['CAC21482.1']
    Key: translation, Value: ['MQTVMMDDIQSTDSIAEKDNHSNNESNFTWKAFREQVEKHFSKIERLHQVLGTDGDNSSLFELFTTAMNAQLHEMEQCQKKLEDDCQQRIDSIRFLVSSLKLTDDTSSLKIESPLIQCLNRLSMVEGQYMAQYDQKLSTIKEMYHKLESYCNRLGSPFVLPDFENSFLSDVSDAFTESLRGRINEAEKEIDARLEVINSFEEEILGLWSELGVEPADVPQYEQLLESHTNRPNDVYVTQELIDQLCKQKEVFSAEKEKRSDHLKSIQSEVSNLWNKLQVSPNEQSQFGDSSNINQENISLWETELE

In [ ]:
# Let's look at the types:

print("feature:", type(ase1_locus.features[8]))
print("feature.type:", type(ase1_locus.features[8].type))
print("feature.location:", type(ase1_locus.features[8].location))
print("feature.qualifiers:", type(ase1_locus.features[8].qualifiers))
print("feature.qualifiers['gene']:", type(ase1_locus.features[8].qualifiers["gene"]))


feature: <class 'Bio.SeqFeature.SeqFeature'>
feature.type: <class 'str'>
feature.location: <class 'Bio.SeqFeature.CompoundLocation'>
feature.qualifiers: <class 'dict'>
feature.qualifiers['gene']: <class 'list'>


This information is important if we want to extract the sequence of a specific feature, or filter a subset of features to do something with them.

In [ ]:
# Extract the dseqrecord of the locus only
# Note how we first check if the key 'gene' is in the qualifiers and then check if the value 'ase1' is in the list of qualifiers['gene'], not equal to "ase1"

locus_feature = next(feature for feature in ase1_locus.features if 'gene' in feature.qualifiers and "ase1" in feature.qualifiers['gene'])

# Then we can get a subset of the dseqrecord like this:
extracted_locus = locus_feature.extract(ase1_locus)
print("length of extracted_locus:", len(extracted_locus))

## Another example, getting a subset of features representing the 3' UTRs
utr_features = [feature for feature in ase1_locus.features if feature.type == "3'UTR"]
print("number of utr_features:", len(utr_features))

length of extracted_locus: 2538
number of utr_features: 3


More details on how to work with features in the [Dseq_Features notebook](./Dseq_Features.ipynb).

## Cloning

pydna has multiple functions to simulate cloning, having support for the following methods:
* Gibson Assembly
* Restriction and Ligation / Golgen gate
* Homologous recombination
* CRISPR-Cas9 (ask the organisers if you plan to use it)
* Gateway Cloning
* Cre-Lox recombination

### Restriction and ligation

There are several ways to do restriction and ligation in pydna. You can first do the restriction, pick the fragments for ligation, then do the ligation. You can also do everything in one step. The second method is relatively new, so most examples in the library use the first method.

Let's do an example in which we clone an insert into a backbone plasmid.


#### Using the first method (first restriction, then ligation)

In [ ]:
## Using the first method
from Bio.Restriction import EcoRI, SalI


# A dummy backbone plasmid, let's assume that we want to replace the
# cccc by our insert.
backbone = Dseqrecord('aaaaaaaGAATTCcccGTCGACaa', circular=True)

restriction_products = backbone.cut([EcoRI, SalI])
# We print the products
print('The first one is the cutout - we discard it')
print(repr(restriction_products[0].seq))
print()
print('The second one is the digested backbone - we keep it')
print(repr(restriction_products[1].seq))

digested_backbone = restriction_products[1]


The first one is the cutout - we discard it
Dseq(-13)
AATTCcccG
    GgggCAGCT

The second one is the digested backbone - we keep it
Dseq(-19)
TCGACaaaaaaaaaG
    GtttttttttCTTAA


Now we do the same with the insert

In [ ]:
insert = Dseqrecord('aaGAATTCaaaccgGTCGACaa')
restriction_products = insert.cut([EcoRI, SalI])

# The second one is the one we want
digested_insert = restriction_products[1]

digested_insert.seq


Dseq(-16)
AATTCaaaccgG
    GtttggcCAGCT

Finally we ligate the products, and then circularize

In [ ]:
# They have to be in the right order, and fragments must be complementary
linear_ligation = digested_insert + digested_backbone

# Then we used the function looped to circularize
# IMPORTANT: this functions does not act in place, it returns a new sequence.

desired_plasmid = linear_ligation.looped()

desired_plasmid.seq

Dseq(o27)
AATTCaaaccgGTCGACaaaaaaaaaG
TTAAGtttggcCAGCTGtttttttttC

#### Using the second method (restriciton-ligation in one step)

This is particularly useful for restriction-ligation reactions with many inputs, for example for Golden Gate.


In [ ]:
from pydna.assembly2 import restriction_ligation_assembly

# We indicate that we only want circular products
products = restriction_ligation_assembly([backbone, insert], [EcoRI, SalI], circular_only=True)

print("The first plasmid is the desired one")
print(repr(products[0].seq))
print()
print("The second one is a circularization of the digested insert and the cutout")
print(repr(products[1].seq))

The first plasmid is the desired one
Dseq(o27)
TCGACaaaaaaaaaGAATTCaaaccgG
AGCTGtttttttttCTTAAGtttggcC

The second one is a circularization of the digested insert and the cutout
Dseq(o21)
AATTCcccGTCGACcggtttG
TTAAGgggCAGCTGgccaaaC


### PCR

Using pydna, you can easily simulate PCR. For details on primer design, visit [the documentation](https://pydna-group.github.io/pydna/markdown_notebooks/PCR.html).

In [ ]:
from pydna.amplify import pcr
from pydna.dseqrecord import Dseqrecord
from pydna.primer import Primer

#Defining forward and reverse primers
fwd_primer = Primer("aaaATTCCTGCAGAGTACATC", id="fwd_primer")
rvs_primer = Primer("cccACCATCCGAAGATATCTT", id="rvs_primer")

template = Dseqrecord("gccATTCCTGCAGAGTACATCaaaccggttAAGATATCTTCGGATGGTatta")

pcr_product = pcr(fwd_primer, rvs_primer, template)
print(pcr_product.seq)
print(pcr_product.figure())


aaaATTCCTGCAGAGTACATCaaaccggttAAGATATCTTCGGATGGTggg
   5ATTCCTGCAGAGTACATC...AAGATATCTTCGGATGGT3
                         ||||||||||||||||||
                        3TTCTATAGAAGCCTACCAccc5
5aaaATTCCTGCAGAGTACATC3
    ||||||||||||||||||
   3TAAGGACGTCTCATGTAG...TTCTATAGAAGCCTACCA5


### Gibson Assembly

See below a dummy example of Gibson Assembly. More details on primer design, etc. in [the documentation examples](https://pydna-group.github.io/pydna/markdown_notebooks/Example_Gibson.html).

Equivalent methods, such as in-fusion and fusion-pcr are also available.

In vivo assembly is also available. The difference is that a Gibson Assembly only joins fragments that have terminal overlaps (at the end of the fragments), but the in vivo assembly (which relies on homologous recombination) can join fragments that have non-terminal overlaps.


In [ ]:
from pydna.assembly2 import gibson_assembly

fragment1 = Dseqrecord("GTCGACTaaaAGAGACC")
fragment2 = Dseqrecord("AGAGACCcgcGTCGACT")


# In the funtion, we specify the fragments and the minimum overlap length,
# for this example, we want a minimum overlap of 7 bp.
products = gibson_assembly([fragment1, fragment2], limit=7)
products[0].seq


Dseq(o20)
GTCGACTaaaAGAGACCcgc
CAGCTGAtttTCTCTGGgcg

### Genome integration / excision

Homologous recombination, Cre-Lox recombination and other methods that are normally used on genomes have special arguments

Insertions take a first `Dseqrecord` object with the genome, and a list of `Dseqrecord` objects with the fragments to integrate / excise.

Excisions take a single `Dseqrecord` object with the genome from which to excise a circular DNA molecule.

Let's take an example for homologous recombination. In which we want to integrate a plasmid into the genome via a single homology site:


In [ ]:
from pydna.assembly2 import homologous_recombination_integration

genome = Dseqrecord('aaaCGTACGCACAaaa')
plasmid = Dseqrecord('CGTACGCACAcccc', circular=True)

products = homologous_recombination_integration(genome, [plasmid], limit=10)

transformed_genome = products[0]
transformed_genome.seq



Dseq(-30)
aaaCGTACGCACAccccCGTACGCACAaaa
tttGCATGCGTGTggggGCATGCGTGTttt

We can also simulate the opposite process, an excission, and recover the original inputs (plasmid and gneome)

In [ ]:
from pydna.assembly2 import homologous_recombination_excision

products = homologous_recombination_excision(transformed_genome, limit=10)

print(products[0].seq)
print(products[1].seq)

CGTACGCACAcccc
aaaCGTACGCACAaaa


### Other cloning methods

You can find most other cloning methods at the bottom of the [documentation page of the assemmbly2 module](https://pydna-group.github.io/pydna/modules/pydna_assembly2.html).

## Sequence equality

Sometimes, you may want to check if two double-stranded sequences are equal. Comparing the sequence strings is not great, because they could be reverse-complemented, and for circular molecules, they might be shifted. Pydna comes with a function that produces a unique string for a given sequence, independently of its orientation or rotation (for circular DNA).

In [ ]:
my_plasmid = Dseqrecord('AAGGTC', circular=True)
my_plasmid_rotated = my_plasmid.shifted(3)
my_plasmid_rotated_rc = my_plasmid_rotated.reverse_complement()

print(my_plasmid.seq, my_plasmid.seguid())
print(my_plasmid_rotated.seq, my_plasmid_rotated.seguid())
print(my_plasmid_rotated_rc.seq, my_plasmid_rotated_rc.seguid())



AAGGTC cdseguid=J56O6n-fwbT2-SLlxCMBVDjXzKg
GTCAAG cdseguid=J56O6n-fwbT2-SLlxCMBVDjXzKg
CTTGAC cdseguid=J56O6n-fwbT2-SLlxCMBVDjXzKg
